In [9]:
import pandas as pd
import sys
import os.path as op

basename = "/home/adrian/PhD/Tweet-Classification-Diabetes-Distress/"
path_utils = op.join(basename , "utils")
sys.path.insert(0, path_utils)

from sys_utils import load_library
from tweet_utils import tweet_vectorizer
from defines import Patterns, Emotions
from emotion_codes import UNICODE_EMOJI, EMOJI_TO_CATEGORY
from preprocess import Preprocess
prep = Preprocess()

data_path = "/home/adrian/PhD/Data/Tweets20210128/Tweets_per_months_personal_noJokes/matching-tweets_diab_noRT-noDupl_20210128_personal_noJokes_withFullText.parquet" 


[nltk_data] Downloading package punkt to /home/adrian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/adrian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/adrian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data
data = pd.read_parquet(data_path)#.sample(n=40, random_state=33)
print(data.columns)
print("Total count:", data.shape[0])
data.head()

Index(['id', 'created_at', 'lang', 'text', 'user_name', 'user_screen_name',
       'user_followers_count', 'user_friends_count', 'user_location',
       'user_description', 'place_full_name', 'retweeted_user_name',
       'retweeted_user_screen_name', 'retweeted_user_followers_count',
       'retweeted_user_friends_count', 'retweeted_user_location',
       'retweeted_user_description', 'retweeted_place_full_name',
       'retweeted_text', 'Project', 'posted_month', 'tweet_URL_USER',
       'full_text'],
      dtype='object')
Total count: 1830536


,id,created_at,lang,text,user_name,user_screen_name,user_followers_count,user_friends_count,user_location,user_description,...,retweeted_user_followers_count,retweeted_user_friends_count,retweeted_user_location,retweeted_user_description,retweeted_place_full_name,retweeted_text,Project,posted_month,tweet_URL_USER,full_text
0,862697033131597824,Thu May 11 15:52:55 +0000 2017,en,Hypo in the Boticelli rooms at the @UffiziGall...,Samantha Morris,TheBorgiaBuIl,624.0,570.0,"Southampton, England",Tattooed Renaissance historian and author. Aut...,...,None,None,None,None,None,None,Diabetes,201705,Hypo in the Boticelli rooms at the USER and on...,Hypo in the Boticelli rooms at the @UffiziGall...
1,862729763969835008,Thu May 11 18:02:59 +0000 2017,en,Richard had his medical paperwork and wallet i...,Against Displacement,stopdisplacemnt,827.0,307.0,occupied & unceded BC,#stopdisplacement,...,None,None,None,None,None,None,Diabetes,201705,Richard had his medical paperwork and wallet i...,Richard had his medical paperwork and wallet i...
2,862733663443615744,Thu May 11 18:18:28 +0000 2017,en,Life &amp; diabetes in a jar altogether! 😍 I'...,Hasan Abughosh,H_Abughosh,2420.0,1455.0,Hashemite Kingdom of Jordan,Marketing Manager of Easy Jordan | Startup & T...,...,None,None,None,None,None,None,Diabetes,201705,Life &amp; diabetes in a jar altogether! 😍 I'...,Life &amp; diabetes in a jar altogether! 😍\n\n...
3,862757080142274561,Thu May 11 19:51:31 +0000 2017,en,Incredible...... Six months ago I was doing 72...,Raymond Tonsing,tonsing,2078.0,270.0,San Francisco,"Working hard @ Caffeinated Capital. Family, te...",...,None,None,None,None,None,None,Diabetes,201705,Incredible...... Six months ago I was doing 72...,"Incredible......\n""Six months ago I was doing ..."
4,862779296292712448,Thu May 11 21:19:48 +0000 2017,en,@parkourstar1234 @Afrezza4Teens @RooksLeAnne n...,Brat81,horan5775,161.0,229.0,None,None,...,None,None,None,None,None,None,Diabetes,201705,USER USER USER not any different than issues m...,@parkourstar1234 @Afrezza4Teens @RooksLeAnne n...


In [11]:
def check_emoticonEmoji_in_tweet(tweet):
    """ Check if in each tweet occurs either an emoji or emoticon """
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.tokenize(tweet)

    for word in tweet: 
        match_emoticon = Patterns.EMOTICONS_PATTERN.findall(word)
        
        # if emoticon found
        if match_emoticon:
            if match_emoticon[0] is not ":" and match_emoticon[0] is not ")": return True
            
        # if emoji found    
        if word in UNICODE_EMOJI: 
            emot_cat = EMOJI_TO_CATEGORY[UNICODE_EMOJI[word]]
            if emot_cat != "": return True

        return False


def check_emotKeywords(tweet):
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = tweet.replace("nerve pain", "")
    tweet = tweet.replace("nerve damage", "")
    tweet = tweet.replace("pain of neuropathy", "")
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = [word for word in tweet if word not in ["healthcare"]]

    for word in tweet:
        for emot in Emotions.emotions_full_list:
            if emot == word: return True
    return False


data["emotion"] = data.full_text.map(lambda tweet: check_emoticonEmoji_in_tweet(tweet) or check_emotKeywords(tweet))

emot_tweets = data[data["emotion"] == True]
print(emot_tweets.shape)
data[["full_text", "emotion"]].head(200)

<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-11-9db2c64d8f35>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if match_emoticon[0] is not ":" and match_emoticon[0] is not ")": return True
<ipython-input-11-9db2c64d8f35>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if match_emoticon[0] is not ":" and match_emoticon[0] is not ")": return True


(562013, 24)


,full_text,emotion
0,Hypo in the Boticelli rooms at the @UffiziGall...,True
1,Richard had his medical paperwork and wallet i...,False
2,Life &amp; diabetes in a jar altogether! 😍\n\n...,True
3,"Incredible......\n""Six months ago I was doing ...",False
4,@parkourstar1234 @Afrezza4Teens @RooksLeAnne n...,False
...,...,...
195,@Mark_Sheppard @DiabetesUK As a type 1 diabeti...,False
196,'serving chicken at a diabetes event is the sa...,False
197,Umm wtf ok US you seriously need a Transgender...,True
198,I had to get my blood drawn todayyyy ugh they ...,True


In [13]:
for i, row in data[["full_text", "emotion"]][0:200].iterrows():
    print(row["full_text"])
    print(row["emotion"])
    print()

Hypo in the Boticelli rooms at the @UffiziGalleries and one of the staff members offered me a sweet cause he knew my pain #type1diabetes
True

Richard had his medical paperwork and wallet in his tent that he needs for his insulin, bylaw is refusing access to it
False

Life &amp; diabetes in a jar altogether! 😍

I'm sorry I don't care about the size of my inflated… https://t.co/ia4Lm9cTVH
True

Incredible......
"Six months ago I was doing 72 units of insulin a day. Now I’m doing none.”
https://t.co/WNytMbFAgw
False

@parkourstar1234 @Afrezza4Teens @RooksLeAnne not any different than issues my child has had between insulin,pump,CGM same issues
False

It's always worrying as a diabetic when your foot starts to hurt. I don't want to become a one footed person
True

@CRichardson1997 finally something that comes close to comparing to the "av got diabetes" vid https://t.co/DaoF4j4KVY
False

No no no no no. Why would a diabetic (t1 or t2) want the roller coaster following this carb bomb? Why h

In [12]:
Emotions.emotions_full_list

['enjoyment',
 'torturous',
 'shameful',
 'affright',
 'abominate',
 'disquietude',
 'beguiled',
 'worry',
 'displeased',
 'piquant',
 'despair',
 'attender',
 'shiver',
 'emotional',
 'restiveness',
 'rejective',
 'benevolent',
 'exulting',
 'rally',
 'sneerer',
 'jealous',
 'attractable',
 'unconscionable',
 'brass',
 'chafe',
 'yeasty',
 'heartbreak',
 'unhappy',
 'nettled',
 'uncomfortableness',
 'teaser',
 'agonise',
 'disturbing',
 'electrifying',
 'zestfully',
 'disposition',
 'staggering',
 'exorbitance',
 'intoxicate',
 'traumatise',
 'turmoil',
 'mirthfully',
 'grieve',
 'hardiness',
 'nervousness',
 'surprising',
 'loathing',
 'acerb',
 'delirium',
 'disappointment',
 'galling',
 'tormenter',
 'embarrassing',
 'ebullience',
 'arouse',
 'uncheerful',
 'isolate',
 'hysteria',
 'offend',
 'tender',
 'drear',
 'fairly',
 'risible',
 'disdain',
 'throb',
 'astound',
 'happiness',
 'brashly',
 'celebration',
 'jounce',
 'queer',
 'envious',
 'unrestrained',
 'raging',
 'ashamed',


In [14]:
emot_tweets.head()

,id,created_at,lang,text,user_name,user_screen_name,user_followers_count,user_friends_count,user_location,user_description,...,retweeted_user_friends_count,retweeted_user_location,retweeted_user_description,retweeted_place_full_name,retweeted_text,Project,posted_month,tweet_URL_USER,full_text,emotion
0,862697033131597824,Thu May 11 15:52:55 +0000 2017,en,Hypo in the Boticelli rooms at the @UffiziGall...,Samantha Morris,TheBorgiaBuIl,624.0,570.0,"Southampton, England",Tattooed Renaissance historian and author. Aut...,...,None,None,None,None,None,Diabetes,201705,Hypo in the Boticelli rooms at the USER and on...,Hypo in the Boticelli rooms at the @UffiziGall...,True
2,862733663443615744,Thu May 11 18:18:28 +0000 2017,en,Life &amp; diabetes in a jar altogether! 😍 I'...,Hasan Abughosh,H_Abughosh,2420.0,1455.0,Hashemite Kingdom of Jordan,Marketing Manager of Easy Jordan | Startup & T...,...,None,None,None,None,None,Diabetes,201705,Life &amp; diabetes in a jar altogether! 😍 I'...,Life &amp; diabetes in a jar altogether! 😍\n\n...,True
5,862832546945675264,Fri May 12 00:51:24 +0000 2017,en,It's always worrying as a diabetic when your f...,­­­Mitth'raw'nuruodo,MattWWeir,257.0,177.0,None,"So please forgive what I have done. No, you ca...",...,None,None,None,None,None,Diabetes,201705,It's always worrying as a diabetic when your f...,It's always worrying as a diabetic when your f...,True
14,863308516198678529,Sat May 13 08:22:44 +0000 2017,en,Wut about iam my own mum people? They gift tan...,Saad,msaadkb,1050.0,567.0,SC:msaadkb,you'll will never know what i am really trying...,...,None,None,None,None,None,Diabetes,201705,Wut about iam my own mum people? They gift tan...,"Wut about ""iam my own mum"" people? They gift t...",True
15,863399191497904130,Sat May 13 14:23:03 +0000 2017,en,"If you have type 1 diabetes , you don't deserv...",NastyNana-demonation,LandauCamar,134.0,219.0,New York,"I am WOMAN , I'm NEW YORK & Jewish PROUD. Wife...",...,None,None,None,None,None,Diabetes,201705,"If you have type 1 diabetes , you don't deserv...","If you have type 1 diabetes , you don't deserv...",True


In [15]:
emot_tweets.to_parquet("/home/adrian/PhD/Data/Tweets20210128/Tweets_per_months_personal_noJokes/matching-tweets_diab_noRT-noDupl_20210128_personal_noJokes_withFullText_emotional.parquet")
